In [1]:
import numpy as np
import h5py
from datetime import datetime

In [2]:
import os

In [3]:
os.environ["MKL_NUM_THREADS"] = "12" 
os.environ["NUMEXPR_NUM_THREADS"] = "10" 
os.environ["OMP_NUM_THREADS"] = "12" 

In [1]:

def nbrs(cur, x, y):

    layer = x * y
    xl = (cur - 1) % x + x * (cur // x)
    xr = (cur + 1) % x + (cur // x) * x
    yl = (cur - x) % layer + layer * (cur // layer)
    yr = (cur + x) % layer + layer * (cur // layer)
    return xl, xr, yl, yr


def make_mtx(x, y, t1):
    N = x * y
    m = np.zeros((N, N))
    for n in range(N):
        ns = nbrs(n, x, y)
        m[n, [ns]] = t1
    return m


def recompute_par(evals, evecs, mu_0, N, U):
    
    indx = np.where(evals >= 0.0)[0]
    delta = np.abs(U) * np.sum(evecs[:N, indx] * np.conj(evecs[N:, indx]), axis=1)
    n_avg = 2 * np.sum(evecs[N:, indx] ** 2, axis=1)     
    mu = mu_0 + 0.5 * np.abs(U) * n_avg
    return (delta, mu, n_avg)


def construct_H(M, N, U, V, mu, delta):
    
    h1 = np.concatenate((M - (mu - V) * np.eye(N), delta * np.eye(N)), axis=0)
    h2 = np.concatenate((delta * np.eye(N), -M + (mu - V) * np.eye(N)), axis=0)
    return np.concatenate((h1, h2), axis=1)


def coordinate(number, a, b):
    return np.array([number % a, number // b])


def distance(i, j, a, b):
    dx, dy = coordinate(i, a, b) - coordinate(j, a, b)
    if np.abs(dx) > a/2:
        dx = a - np.abs(dx)
    if np.abs(dy) > b/2:
        dy = b - np.abs(dy)
    return np.linalg.norm([dx,dy])
    #return coordinate(i, a, b) - coordinate(j, a, b))

def dd(n, x, y, delta):
    data = {}
    for i in range(n):
        for j in range(n):
            rij = distance(i, j, x, y)
            if rij not in data:
                data[rij] = [delta[i] * delta[j]]
            else:
                data[rij].append(delta[i] * delta[j])
    return data

def write(file, e, v, delta, n):
    with h5py.File(file) as f:
        f.create_dataset("evals", data=e)
        f.create_dataset("evecs", data=v)
        f.create_dataset("delta", data=delta)
        f.create_dataset("avg", data=n)

def run(U, M, N, V, delta, max_iterations, tol, mu_0, r, l, point, file):
    
    avg = 0
    while np.abs(point - avg) > tol:
        print(f"{np.round(np.abs(point - avg), 4)}", end = ' :')
        mu = mu_0 + 0.5 * np.abs(U) * np.random.sample(N)
        atol = 10
        step = 0
        while atol > 1e-08 and step < max_iterations:
            
            H = construct_H(M, N, U, V, mu, delta)
            e, v = np.linalg.eigh(H)
            delta, mu_new, avg = recompute_par(e, v, mu_0, N, U)
            atol = np.linalg.norm(mu_new - mu)
            mu = mu_new
            step += 1 
#         print(f'<n>={np.sum(avg)/N} mu={mu_0}')
        avg_dist = avg
        avg = np.sum(avg)/N
        if point - avg > 0:
            l = mu_0
        else:
            r = mu_0
        mu_0 = 0.5 * (r + l)

    write(file, e, v, delta, avg_dist)

Writing BdG.py


In [5]:
M = make_mtx(8, 8, -1)
N = M.shape[0]
V0 = 0.5
V = V0 * (2 * np.random.sample(N) - 1) 
dirp = f"data_V{V0:.2f}"
for U in [2]:
    print('U = {}'.format(U), end=' : ')
    delta = - np.abs(U) * np.random.sample(N)
    max_iterations =  2000
    tol = 1e-03
    mu_0 = -0.9
    r = 0
    l =-10
    point = 0.125
    
    if not os.path.isdir(dirp):
        os.mkdir(dirp)
    file = f"{dirp}/U{U}_point{point}"
    t1 = datetime.now()
    run(U, M, N, V, delta, max_iterations, tol, mu_0, r, l, point, file)
    t2 = datetime.now()
#         data = dd(24 ** 2, 24, 24, delta)
    print(':: Loop time {}'.format((t2 - t1).total_seconds()))

U = 2 : 0.125 :0.9632 :0.125 :0.0313 :0.125 :0.0937 :0.0312 :0.0747 :0.0141 :0.0151 :0.0011 :0.0064 :0.0026 ::: Loop time 0.721338


/home/titan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:61: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
